In [ ]:
#Import dependencies 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Read the csv files
olympics_df = pd.read_csv('Resources/athlete_events.csv')
regions_df = pd.read_csv('noc_regions.csv')
cities_df = pd.read_csv('worldcities.csv')
arrivals_df = pd.read_csv('Resources/INT.ARVL.csv')
departures_df = pd.read_csv('Resources/INT.DPRT.csv')
expenditures_df = pd.read_csv('Resources/INT.XPND.csv')

In [ ]:
#Jumaan start here (Question 1)

In [ ]:
#Jumaan end here

In [ ]:
#Timmy start here (Question 2)

In [ ]:
#Timmy end here

In [ ]:
#Raj start here (Question 3)

In [ ]:
#Raj end here

In [ ]:
#Fatma start here (Question 4)

In [ ]:
#Fatma end here